In [41]:
%pip install -qU pip
%pip install -qU sagemaker boto3 awscli boto3 ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [42]:
import json
from pathlib import Path
import os
import sys
from time import strftime
from functools import partial
import importlib

import boto3
import sagemaker
from sagemaker.experiments.run import Run
from sagemaker.inputs import TrainingInput
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [43]:
sagemaker.__version__, boto3.__version__

('2.233.0', '1.35.60')

In [4]:
import os
print(os.getcwd())

/home/ec2-user/SageMaker/notebooks


In [6]:
!bash ../scripts/mount_s3.sh

--2024-11-13 22:00:12--  https://s3.amazonaws.com/mountpoint-s3-release/latest/x86_64/mount-s3.rpm
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.192.239, 52.217.235.56, 52.217.98.158, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.192.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12266864 (12M) [binary/octet-stream]
Saving to: ‘/home/ec2-user/mount-s3.rpm’

100%[======================================>] 12,266,864  13.2MB/s   in 0.9s   

2024-11-13 22:00:13 (13.2 MB/s) - ‘/home/ec2-user/mount-s3.rpm’ saved [12266864/12266864]

package mount-s3 is not installed
Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock
Examining /home/ec2-user/mount-s3.rpm: mount-s3-1.10.0-1.x86_64
Marking /home/ec2-user/mount-s3.rpm to be installed
Resolving Dependencies
--> Running transaction check
---> Package mount-s3.x86_64 0:1.10.0-1 will be installed
--> Processing

In [10]:
# Check the mount point exists
!ls -la ../s3/newatlantis-raw-veba-db-prod

# Try listing with hidden files
!ls -la ../s3/newatlantis-raw-veba-db-prod/.* 2>/dev/null

# Check mount status
!mount | grep s3

total 8
drwxrwxr-x 2 ec2-user ec2-user 4096 Nov 13 22:00 .
drwxrwxr-x 3 ec2-user ec2-user 4096 Nov 13 22:00 ..
../s3/newatlantis-raw-veba-db-prod/.:
total 8
drwxrwxr-x 2 ec2-user ec2-user 4096 Nov 13 22:00 .
drwxrwxr-x 3 ec2-user ec2-user 4096 Nov 13 22:00 ..

../s3/newatlantis-raw-veba-db-prod/..:
total 12
drwxrwxr-x  3 ec2-user ec2-user 4096 Nov 13 22:00 .
drwxr-xr-x 11 ec2-user ec2-user 4096 Nov 13 22:00 ..
drwxrwxr-x  2 ec2-user ec2-user 4096 Nov 13 22:00 newatlantis-raw-veba-db-prod


In [8]:
# Check if you have read permissions
!aws s3 ls s3://newatlantis-raw-veba-db-prod/

# Check your AWS credentials are working
!aws sts get-caller-identity

                           PRE MicroEuk_v3/
                           PRE UniRef/
                           PRE VDB_v5.1/
                           PRE VDB_v7/
                           PRE VDB_v8.1/
                           PRE uncompressed/
{
    "UserId": "AROAQYAUD2HFU3H3HHOAN:SageMaker",
    "Account": "051581800907",
    "Arn": "arn:aws:sts::051581800907:assumed-role/AmazonSageMaker-ExecutionRole-20240617T145283/SageMaker"
}


In [14]:
!ls -l /home/ec2-user/SageMaker/s3/newatlantis-raw-veba-db-prod/VDB_v8.1/

ls: cannot access /home/ec2-user/SageMaker/s3/newatlantis-raw-veba-db-prod/VDB_v8.1/: No such file or directory


In [39]:
!aws s3 ls s3://newatlantis-raw-veba-db-prod/VDB_v8.1/Classify/GTDB/skani/database/GCA/000/063/505/

2024-11-08 19:44:32     694354 GCA_000063505.1_genomic.fasta.gz


In [44]:
# This code sets up a SageMaker session and establishes an AWS Omics client

boto_session = boto3.session.Session() # The boto session required to establish a connection to the AWS Omics client
sagemaker_session = sagemaker.session.Session(boto_session) # This line sets up Amazon SageMaker resource interaction (training, model deployment, and experiement control)
omics = boto3.client("omics") # initializing the boto3 client specifically for the AWS Omics service

REGION_NAME = sagemaker_session.boto_region_name # SageMaker resource region extraction (important that the resources being used for training tasks are created in the intended region)
S3_BUCKET = sagemaker_session.default_bucket() # We initialize the default S3 bucket to store data related to SageMaker (model artifacts, datasets, etc..)

EXPERIMENT_NAME = "hyenaDNA-pretraining-v2"

SAGEMAKER_EXECUTION_ROLE = sagemaker.session.get_execution_role(sagemaker_session) # Retrieves the execution role assumed by SageMaker (the role that allows SageMaker to access other AWS resources on behalf of the user)
print(f"Assumed SageMaker role is {SAGEMAKER_EXECUTION_ROLE}")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Assumed SageMaker role is arn:aws:iam::051581800907:role/service-role/AmazonSageMaker-ExecutionRole-20240617T145283


In [45]:
ref_store_id = "8721910603"


In [46]:
response = omics.list_references(
    referenceStoreId=ref_store_id
)
references = response.get('references', [])
print(f"There are {len(references)} references in the reference store.")

There are 50 references in the reference store.


In [48]:
ref_metadata = omics.get_reference_metadata(
    id='6299798618',
    referenceStoreId=ref_store_id
)

In [53]:
ref_metadata['files']

{'source': {'totalParts': 1, 'partSize': 104857600, 'contentLength': 2396626},
 'index': {'totalParts': 1, 'partSize': 104857600, 'contentLength': 28}}

In [55]:
ref_01 = omics.get_reference(
    id='6299798618',
    referenceStoreId=ref_store_id,
    partNumber = 1
)

In [57]:
ref_01['payload']

In [ ]:
seq_store_id = "4789217446"  # replace with your sequence store id

# This cell retrieves the metadata from the sequence store associated with the input ID

seq_store_info = omics.get_sequence_store(id=seq_store_id) # Information includes storage location, access configurations, encryption settings, etc..
s3_uri = seq_store_info["s3Access"]["s3Uri"] # The path where the sequence data is stored (our data pointer)
s3_arn = seq_store_info["s3Access"]["s3AccessPointArn"] # Restricted access pointer for the data sequence store
key_arn = seq_store_info["sseConfig"]["keyArn"] # Encryption key for the sequence data in S3 bucket
s3_uri, s3_arn, key_arn

S3_DATA_URI = f"{s3_uri}readSet/"
S3_DATA_URI

In [ ]:
print(json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "S3DirectAccess",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "s3:DataAccessPointArn": s3_arn
                }
            }
        },
        {
            "Sid": "DefaultSequenceStoreKMSDecrypt",
            "Effect": "Allow",
            "Action": "kms:Decrypt",
            "Resource": key_arn
        }
    ]
}, indent=2))

In [ ]:
# Pointer to the docker image for training config

pytorch_image_uri = f"763104351884.dkr.ecr.{REGION_NAME}.amazonaws.com/pytorch-training:2.2.0-gpu-py310-cu121-ubuntu20.04-sagemaker"
pytorch_image_uri

In [ ]:
# In this cell, we set up the parameters for a training job to fine tune the hyenaDNA model on our dataset of choice


MODEL_ID = 'LongSafari/hyenadna-small-32k-seqlen-hf' # The path / id of the pre-trained model we seek to fine tune
TRAINING_JOB_NAME = 'hyenaDNA-pretraining' # Our job identification label

# Additional training parameters

hyperparameters = {
    "species" : "mouse",
    "epochs": 150,
    "model_checkpoint": MODEL_ID,
    "max_length": 32_000,
    "batch_size": 4,
    "learning_rate": 6e-4,
    "weight_decay" : 0.1,
    "log_level" : "INFO",
    "log_interval" : 100
}


In [ ]:

# The model performance metrics we want SageMaker to monitor

metric_definitions = [
    {"Name": "epoch", "Regex": "Epoch: ([0-9.]*)"},
    {"Name": "step", "Regex": "Step: ([0-9.]*)"},
    {"Name": "train_loss", "Regex": "Train Loss: ([0-9.e-]*)"},
    {"Name": "train_perplexity", "Regex": "Train Perplexity: ([0-9.e-]*)"},
    {"Name": "eval_loss", "Regex": "Eval Average Loss: ([0-9.e-]*)"},
    {"Name": "eval_perplexity", "Regex": "Eval Perplexity: ([0-9.e-]*)"}
]

In [ ]:
# hyenaDNA model training config

hyenaDNA_estimator = PyTorch(
    base_job_name=TRAINING_JOB_NAME,                                      # The job name for organization we assigned earlier
    entry_point="train_hf_accelerate.py",                                 # The python script we will run for training
    source_dir="scripts/",                                                # Reference to directory with all necessary scripts for training
    instance_type="ml.g5.12xlarge",                                       # Compute instance type specification
    instance_count=1,                                                     # Number of compute instances to use for training
    image_uri=pytorch_image_uri,                                          # Inputting the pre-configured Docker image with PyTorch and CUDA dependencies installed
    role=SAGEMAKER_EXECUTION_ROLE,                                        # Specifying the permission levels of SageMaker
    hyperparameters=hyperparameters,                                      # Input the hyperparameter specifications we made earlier
    metric_definitions=metric_definitions,                                # The custom pattern we made earlier specifying what performance metrics to monitor
    sagemaker_session=sagemaker_session,                                  # The session established earlier
    distribution={"torch_distributed": {"enabled": True}},                # Enabling distributed training (letting PyTorch tap into multiple compute resources in parallel for faster training)
    tags=[{"Key": "project", "Value": "genomics-model-pretraining"}],     # Organization
    keep_alive_period_in_seconds=1800,                                    # Allows us to terminate the training instance if the instance is running idle
    tensorboard_output_config=tensorboard_output_config,                  # Specifies the TensorBoard logs to be synced to S3 for real-time training monitoring
)


In [ ]:
# Initiating the SageMaker experiment run to tune the hyenaDNA model

with Run(
    experiment_name=EXPERIMENT_NAME,
    sagemaker_session=sagemaker_session,
) as run:
    hyenaDNA_estimator.fit(
        {
            "data": TrainingInput(
                s3_data=S3_DATA_URI, input_mode="File"                # Here the model training is officially launched! :D
            ),
        },
        wait=True,
    )

In [ ]:
# Pull the latest training job's ID for ease of reference

training_job_name = hyenaDNA_estimator.latest_training_job.name
training_job_name

In [ ]:
user_profile = "shamika" # replace with your sagemaker studio profile name

In [ ]:
# Launches TensorBoard in SageMaker

from sagemaker.interactive_apps.tensorboard import TensorBoardApp

with open("/opt/ml/metadata/resource-metadata.json", "r") as f:
    app_metadata = json.loads(f.read())
    sm_user_profile_name = app_metadata["SpaceName"]
    sm_domain_id = app_metadata["DomainId"]

tb_app = TensorBoardApp(REGION_NAME)
tb_app.get_app_url(
    training_job_name=training_job_name,
    create_presigned_domain_url=True,
    domain_id=sm_domain_id,
    user_profile_name=user_profile,
    open_in_default_web_browser=False,
    optional_create_presigned_url_kwargs={}
)

In [ ]:

from sagemaker.pytorch.model import PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.estimator import Estimator

training_job_name = "hyenaDNA-pretraining-2024-04-06-06-23-26-412"
#attached_estimator = Estimator.attach(training_job_name)

model_data = attached_estimator.model_data
model_data

In [ ]:
# Deploy the model to create a real-time endpoint

endpoint_name = 'hyenaDNA-pretrained-mouse-ep'                 # The name reference for the endpoint we will use to access the model's real-time predictions
pytorch_deployment_uri = f"763104351884.dkr.ecr.{REGION_NAME}.amazonaws.com/pytorch-inference:2.2.0-gpu-py310-cu118-ubuntu20.04-sagemaker"


# Below we use the endpoint definition and the loaded trained model artifacts to intialize / prepare our PyTorchModel for deployment!
hyenaDNAModel = PyTorchModel(
    model_data=model_data,
    role=SAGEMAKER_EXECUTION_ROLE,
    image_uri=pytorch_deployment_uri,
    entry_point="inference.py",
    source_dir="scripts/",
    sagemaker_session=sagemaker_session,
    name=endpoint_name,
    env = {
        'MMS_MAX_REQUEST_SIZE': '2000000000',
        'MMS_MAX_RESPONSE_SIZE': '2000000000',
        'MMS_DEFAULT_RESPONSE_TIMEOUT': '900',
        'TS_MAX_RESPONSE_SIZE':'2000000000',
        'TS_MAX_REQUEST_SIZE':'2000000000',
    }
)

In [ ]:
#real_time_endpoint_name = "hyenaDNA-mouse-pretrained-real-ep"
env = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'7200',
    'TS_MAX_RESPONSE_SIZE':'2000000000',
    'TS_MAX_REQUEST_SIZE':'2000000000',
    'MMS_MAX_RESPONSE_SIZE':'2000000000',
    'MMS_MAX_REQUEST_SIZE':'2000000000'
}

# deploy the endpoint
realtime_predictor = hyenaDNAModel.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.8xlarge",
    endpoint_name=endpoint_name,
    env=env,
)

In [ ]:

# Loading a sample of genomic data

import json
sample_genome_data = []
with open("./sample_mouse_data.json") as file:
    for line in file:
        sample_genome_data.append(json.loads(line))
len(sample_genome_data)

In [ ]:
len(sample_genome_data[0])

In [ ]:

# Retrieves the embedding output of the loaded data on the deployed model

from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer


data = [sample_genome_data[0]]
realtime_predictor.serializer = JSONSerializer()
realtime_predictor.deserializer = JSONDeserializer()
embeddings = realtime_predictor.predict(data=data)

In [ ]:
embeddings

In [ ]:

# Terminating the endpoint access, freeing up resources and stopping SageMaker charges

realtime_predictor.delete_endpoint()